In [ ]:
%%bash
# Script for node 1. SSH via UI Console

# Initialize Vault
vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)

# Save info in GCP Secrets
gcloud secrets create root_token --replication-policy="automatic"
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token --data-file=-
gcloud secrets create vault_init_data --replication-policy="automatic"
cat output.json | gcloud secrets versions add vault_init_data --data-file=-

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# Create Prometheus token and update Ops Agent Configuration with Token
vault policy write prometheus-metrics /opt/vault/prometheus.hcl
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/google-cloud-ops-agent/config.yaml
sudo systemctl restart google-cloud-ops-agent

# Enable Dead Server clean-up, min-quorum should be set in accordance to cluster size
vault operator raft autopilot set-config -cleanup-dead-servers=true -dead-server-last-contact-threshold=1m -min-quorum=3

In [ ]:
%%bash
# Script for node 2-to-N. SSH via UI Console

# Obtain root token
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token)

# Create Prometheus token and update Ops Agent Configuration with Token
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/google-cloud-ops-agent/config.yaml
sudo systemctl restart google-cloud-ops-agent

In [ ]:
%%bash
# from your terminal
# Enable automatic snapshots
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token --project=<your-project-id>)
gcloud iam service-accounts keys create sa-keys.json --iam-account=$(terraform output -raw sa-name)
VAULT_ADDR=$(terraform output -json | jq -r .fqdn_8200.value) vault write sys/storage/raft/snapshot-auto/config/hourly interval="1h" retain=10 path_prefix="snapshots/" storage_type=google-gcs google_gcs_bucket=$(terraform output -raw gcs_bucket_snapshot) google_service_account_key="@sa-keys.json"